In [13]:
import numpy as np
import networkx as nx
import random

In [12]:
# calculate answers of Question 1
# average degree of neighbors
(10**(2.1)-1)/(-1*(10**(-0.9)-1))

142.88007717944873

In [11]:
# calculate answers of Question 1
# average degree of a node
((10**(-0.9)-1)/(10**(-3.9)-1))*(1.3/0.3)

3.7882759039027594

In [14]:
G = nx.complete_graph(4)

In [22]:
G.nodes()

NodeView((0, 1, 2, 3))

In [28]:
len(G.edges())

6

In [40]:
degrees = []
nodes = G.nodes()
for node in nodes:
    # print(G.degree(node))
    # print(node)
    degrees.append(G.degree(node)/(2*len(G.edges())))
#degrees=[0,0,0,1]
print(np.random.choice(G.nodes(),p=degrees))


3


In [ ]:
def run_experiment(No, N, m):
    # start with a No node fully connected graph
    G = nx.complete_graph(No)
    # add nodes
    for i in range(No+1, N):
        
